# Testgroud for OWR project

importing modules for data analisis and creating predition models

In [ ]:
from cv2 import data, imread, resize
from keras import models
from numpy.lib.histograms import histogram
from tqdm import tqdm, utils
from random import shuffle
from numpy import array, log
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import callbacks, utils

importing python built in modules and those which are not related to data analisis

In [ ]:
from os.path import join, dirname, abspath
from os import error, listdir, path
from datetime import datetime

initialization,  here the array size and location for logs are configured
images size have to be equal determined during function initialization.
<br>
Variables:
<br>
`path: string` - path to the directory with images <br>
`size: tuple` - tuple describing size of each image <br>
`logDir: string` - directory where run logs are saved

In [ ]:
class ImageRecgonition:
    def __init__(self, path: str, size: tuple, logDir: str = "logs\\fit\\"):
        self.path = path
        self.size = size
        self.logDir = f"{logDir}log_{datetime.now().strftime('%Y%m%d-%H%M%S')}"


Dataset defining takes no arguments but self. It builds an array structure for categorical crossentropy to process data. Number of categories is defined by number of directories inside data folder.

In [ ]:
    def define_dataset(self) -> list:

        self.dataset: list = []
        self.categories = [ d for d in listdir(join(dirname(abspath(__file__)), self.path)) ]   # look for all subdirectories and name categories from them 
        # what the fuck? vvv
        for cat in self.categories:
            imgPath: str = join(join(dirname(abspath(__file__)), self.path), cat)
            categoryIndex: int = self.categories.index(cat)
            for img in tqdm(listdir(path)):
                try:
                    imgArray: list = imread(join(path, img))
                    actualImg = resize(imgArray, self.size)
                    self.dataset.append([actualImg, categoryIndex])
                except Exception as e:
                    exit(f"Error in 'ImageRecoginiton.define_dataset' occured - stopped working: {e}")
        # end of what the fuck? ^^^


Function parameters: <br>
`batchSize: int` - size of the batch (amount) of files processed at the same time by the machine <br>
`epochs: int` - number of times the shell has to be re-taught and how many iterations of script learning have to be done <br> 
`validationSplit: float` - percentage of images that are used to validate learining process. Images are chosen randomly and they are substracted from the main pool

In [ ]:
    def modeling(
        self, 
        batchSize: int,
        epochs: int,
        validationSplit: float):

        shuffle(self.dataset)
        trainingX, labelsY = [], []

        for features, label in self.dataset:    # preparing dataset and its labels for further modeling
            trainingX.append(features)
            labelsY.append(label)
        
        trainingX: array = array(trainingX).reshape(-1, self.size[0], self.size[1], 3)  # reshape data as a numpy array
        trainingX = trainingX/255.0

        # actual modelingg, adding layers and creating models
        self.modelSequential = Sequential()
        
        self.modelSequential.add(Conv2D(256, (3, 3), input_shape=trainingX.shape[1:]))
        self.modelSequential.add(Activation('relu'))
        self.modelSequential.add(MaxPooling2D(pool_size=(2, 2)))

        self.modelSequential.add(Conv2D(256, (3, 3)))
        self.modelSequential.add(Activation('relu'))
        self.modelSequential.add(MaxPooling2D(pool_size=(2, 2)))

        self.modelSequential.add(Flatten()).add(Dense(64)).add(Dense(7))
        self.modelSequential.add(Activation('sigmoid'))
        # TODO modeling in beta version
        # TODO create several models to choose from as methods

        self.modelSequential.compile(
            loss="categorical_crossentropy", 
            optimizer="adam", 
            metrics=["accutracy"])      # compiling model defined above
        labelsYCategorical = utils.to_categorical(labelsY, num_classes=7)     # converting labelsY as a categories

        tensorboard = callbacks.TensorBoard(log_dir = self.logDir, histogram_freq = 1)      # creating tensorboard

        try:
            self.modelSequential.fit(
                trainingX, 
                labelsYCategorical, 
                batch_size = batchSize, 
                epochs = epochs, 
                validation_split = validationSplit,
                callbacks = [tensorboard])
        except Exception as e:
            print(f"Error unhandled: {e} while trying to fit model in 'ImageRecgonition.modeling")
            self.modelSequential.fit(
                trainingX, 
                labelsYCategorical, 
                batch_size = batchSize, 
                epochs = epochs, 
                validation_split = validationSplit)


function is calling predicting moel and validates it's accuracy at the end.
Parameters: <br>
`model` - model of neural network created to test application <br>
`testIMG` - map of bits in PNG/WEBP/JPG format of a given size.

In [ ]:
    def calling_predictions(self, model, testImg):          # predicting tests
        # preparing models and dimensions
        imgArray = imread(testImg)
        actualImg = resize(imgArray, self.size).reshape(-1, self.size[0], self.size[1], 3)
        prediction = model.predict([actualImg])      # pick a test sample
        return self.categories[int(prediction[0][0])]


image size (resoltion) error hanlding

In [ ]:
    def size(self) -> int:
        try:
            return len(self.dataset)
        except UnboundLocalError as e:
            return f"Error in 'ImageRecognition.size' - dataset not created yet: {e}. Use 'ImageRecognition.define_dataset'"
        except error as e:
            return f"Error in 'ImageRecognition.size' - unhandled error: {e}"
